# Test LSTM Food Freshness Model

This notebook loads the trained ONNX model and tests it with sample sensor data.

In [1]:
# Install required packages if needed
!pip install onnxruntime pandas numpy scikit-learn

ERROR: Could not find a version that satisfies the requirement onnxruntime (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Asus\AppData\Local\Programs\Python\Python314\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for onnxruntime


In [ ]:
import numpy as np
import pandas as pd
import onnxruntime as ort
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
# --- Configuration ---
ONNX_MODEL_PATH = "lstm_food_freshness.onnx"
DATA_FILE = "food_freshness_dataset.csv"
SEQUENCE_LENGTH = 10  # Must match training configuration

# Sensor columns (3 analog sensors)
SENSOR_COLUMNS = ['MQ135_Analog', 'MQ3_Analog', 'MiCS5524_Analog']

In [ ]:
# --- Load ONNX Model ---
print(f"Loading ONNX model from {ONNX_MODEL_PATH}...")
ort_session = ort.InferenceSession(ONNX_MODEL_PATH)

# Get model input details
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

print(f"Model loaded successfully!")
print(f"Input name: {input_name}")
print(f"Output name: {output_name}")
print(f"Expected input shape: (batch_size, {SEQUENCE_LENGTH}, {len(SENSOR_COLUMNS)})")

In [ ]:
# --- Load and Prepare Test Data ---
print(f"\nLoading test data from {DATA_FILE}...")
df = pd.read_csv(DATA_FILE)
print(f"Loaded {len(df)} rows")
print(f"Columns: {df.columns.tolist()}")

# Display sample data
print("\nFirst 5 rows:")
print(df.head())

# Display value distribution
print("\nOutput distribution:")
print(df['Output'].value_counts())

In [ ]:
# --- Prepare Scaler (fit on all data for testing purposes) ---
X = df[SENSOR_COLUMNS].values
y = df['Output'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nData scaled. Shape: {X_scaled.shape}")
print(f"Feature means: {scaler.mean_}")
print(f"Feature std devs: {scaler.scale_}")

In [ ]:
# --- Function to Create Sequences ---
def create_test_sequence(data, start_idx, seq_length=10):
    """
    Create a single sequence from the data starting at start_idx.
    Returns shape (1, seq_length, num_features) for model input.
    """
    if start_idx + seq_length > len(data):
        raise ValueError(f"Not enough data. Need {seq_length} samples from index {start_idx}")
    
    sequence = data[start_idx:start_idx + seq_length]
    # Reshape to (1, seq_length, num_features) for batch size of 1
    return sequence.reshape(1, seq_length, -1).astype(np.float32)

# --- Function to Make Prediction ---
def predict(sequence):
    """
    Make prediction using ONNX model.
    Returns probability and class label.
    """
    ort_inputs = {input_name: sequence}
    ort_outputs = ort_session.run(None, ort_inputs)
    probability = ort_outputs[0][0][0]
    prediction = 1 if probability > 0.5 else 0
    return probability, prediction

## Test with Random Samples

In [ ]:
# --- Test with Random Samples ---
print("\n" + "="*60)
print("Testing with Random Samples from Dataset")
print("="*60)

num_tests = 5
np.random.seed(42)

for i in range(num_tests):
    # Pick random starting point (ensure we have enough data for sequence)
    start_idx = np.random.randint(0, len(X_scaled) - SEQUENCE_LENGTH)
    
    # Get the actual label at the end of this sequence
    actual_label = y[start_idx + SEQUENCE_LENGTH - 1]
    
    # Create sequence
    test_sequence = create_test_sequence(X_scaled, start_idx, SEQUENCE_LENGTH)
    
    # Make prediction
    probability, prediction = predict(test_sequence)
    
    # Display results
    print(f"\nTest {i+1}:")
    print(f"  Sequence indices: [{start_idx}:{start_idx + SEQUENCE_LENGTH}]")
    print(f"  Actual Label: {int(actual_label)} ({'Fresh' if actual_label == 1 else 'Bad'})")
    print(f"  Predicted: {prediction} ({'Fresh' if prediction == 1 else 'Bad'})")
    print(f"  Probability (Fresh): {probability:.4f}")
    print(f"  Correct: {'✓' if prediction == actual_label else '✗'}")

## Test with Custom Sensor Values

In [ ]:
# --- Test with Custom Sensor Values ---
print("\n" + "="*60)
print("Testing with Custom Sensor Values")
print("="*60)

# Example 1: Fresh food readings (low analog values)
fresh_readings = np.array([
    [150, 120, 180],  # MQ135, MQ3, MiCS5524
    [145, 125, 175],
    [160, 130, 185],
    [155, 128, 180],
    [148, 122, 178],
    [152, 127, 182],
    [158, 131, 188],
    [151, 124, 179],
    [149, 126, 181],
    [153, 129, 183]
])

# Scale the custom data
fresh_scaled = scaler.transform(fresh_readings)
fresh_sequence = fresh_scaled.reshape(1, SEQUENCE_LENGTH, -1).astype(np.float32)

# Predict
prob, pred = predict(fresh_sequence)
print("\nTest Case 1: Fresh Food (Low Gas Levels)")
print(f"  Sensor readings (avg): MQ135={fresh_readings[:, 0].mean():.1f}, MQ3={fresh_readings[:, 1].mean():.1f}, MiCS5524={fresh_readings[:, 2].mean():.1f}")
print(f"  Prediction: {pred} ({'Fresh' if pred == 1 else 'Bad'})")
print(f"  Probability (Fresh): {prob:.4f}")

# Example 2: Spoiled food readings (high analog values)
spoiled_readings = np.array([
    [650, 720, 680],  # MQ135, MQ3, MiCS5524
    [670, 740, 695],
    [690, 760, 710],
    [680, 750, 700],
    [675, 735, 690],
    [685, 755, 705],
    [695, 770, 715],
    [688, 748, 698],
    [677, 742, 692],
    [683, 752, 703]
])

# Scale the custom data
spoiled_scaled = scaler.transform(spoiled_readings)
spoiled_sequence = spoiled_scaled.reshape(1, SEQUENCE_LENGTH, -1).astype(np.float32)

# Predict
prob, pred = predict(spoiled_sequence)
print("\nTest Case 2: Spoiled Food (High Gas Levels)")
print(f"  Sensor readings (avg): MQ135={spoiled_readings[:, 0].mean():.1f}, MQ3={spoiled_readings[:, 1].mean():.1f}, MiCS5524={spoiled_readings[:, 2].mean():.1f}")
print(f"  Prediction: {pred} ({'Fresh' if pred == 1 else 'Bad'})")
print(f"  Probability (Fresh): {prob:.4f}")

## Visualize Predictions on Sample Data

In [ ]:
# --- Batch Prediction for Visualization ---
print("\n" + "="*60)
print("Running Batch Predictions for Visualization")
print("="*60)

# Test on first 100 sequences
num_samples = min(100, len(X_scaled) - SEQUENCE_LENGTH)
predictions = []
actuals = []
probabilities = []

for i in range(num_samples):
    test_seq = create_test_sequence(X_scaled, i, SEQUENCE_LENGTH)
    prob, pred = predict(test_seq)
    actual = y[i + SEQUENCE_LENGTH - 1]
    
    predictions.append(pred)
    actuals.append(actual)
    probabilities.append(prob)

# Calculate accuracy
correct = sum(p == a for p, a in zip(predictions, actuals))
accuracy = correct / len(predictions) * 100

print(f"\nTested {num_samples} sequences")
print(f"Accuracy: {accuracy:.2f}% ({correct}/{len(predictions)})")

In [ ]:
# --- Visualization ---
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Plot 1: Predictions vs Actuals
axes[0].plot(actuals, 'b-', label='Actual', alpha=0.7, linewidth=2)
axes[0].plot(predictions, 'r--', label='Predicted', alpha=0.7, linewidth=2)
axes[0].set_xlabel('Sample Index')
axes[0].set_ylabel('Class (0=Bad, 1=Fresh)')
axes[0].set_title('Predictions vs Actual Labels')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.1, 1.1])

# Plot 2: Prediction Probabilities
colors = ['red' if p < 0.5 else 'green' for p in probabilities]
axes[1].scatter(range(len(probabilities)), probabilities, c=colors, alpha=0.6, s=20)
axes[1].axhline(y=0.5, color='black', linestyle='--', label='Decision Threshold')
axes[1].set_xlabel('Sample Index')
axes[1].set_ylabel('Probability (Fresh)')
axes[1].set_title('Model Confidence (Green=Fresh, Red=Bad)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

print(f"\nVisualization complete!")

## Test Single Reading Function (Arduino-like)

This simulates how you would use the model with real-time sensor readings from Arduino.

In [ ]:
# --- Simulate Real-Time Sensor Reading Buffer ---
class SensorBuffer:
    """
    Maintains a rolling buffer of sensor readings for LSTM prediction.
    This simulates what you'd implement on Arduino/microcontroller.
    """
    def __init__(self, buffer_size=10, scaler=None):
        self.buffer_size = buffer_size
        self.buffer = []
        self.scaler = scaler
    
    def add_reading(self, mq135, mq3, mics5524):
        """Add new sensor reading to buffer."""
        reading = [mq135, mq3, mics5524]
        self.buffer.append(reading)
        
        # Keep only last buffer_size readings
        if len(self.buffer) > self.buffer_size:
            self.buffer.pop(0)
    
    def is_ready(self):
        """Check if buffer has enough readings for prediction."""
        return len(self.buffer) >= self.buffer_size
    
    def get_sequence(self):
        """Get scaled sequence ready for prediction."""
        if not self.is_ready():
            return None
        
        # Convert to numpy array
        sequence = np.array(self.buffer[-self.buffer_size:])
        
        # Scale using fitted scaler
        if self.scaler:
            sequence = self.scaler.transform(sequence)
        
        # Reshape for model input
        return sequence.reshape(1, self.buffer_size, -1).astype(np.float32)

# Test the buffer
print("\n" + "="*60)
print("Testing Real-Time Sensor Buffer")
print("="*60)

buffer = SensorBuffer(buffer_size=SEQUENCE_LENGTH, scaler=scaler)

# Simulate adding readings one by one (like Arduino loop)
print("\nSimulating real-time sensor readings...\n")

# Fresh food simulation
print("Scenario: Fresh food (low gas levels)")
for i in range(12):  # Add 12 readings
    # Simulate fresh food readings with slight variation
    mq135 = np.random.randint(140, 160)
    mq3 = np.random.randint(120, 135)
    mics5524 = np.random.randint(175, 190)
    
    buffer.add_reading(mq135, mq3, mics5524)
    print(f"  Reading {i+1}: MQ135={mq135}, MQ3={mq3}, MiCS5524={mics5524}")
    
    # Try to predict once buffer is ready
    if buffer.is_ready():
        sequence = buffer.get_sequence()
        prob, pred = predict(sequence)
        print(f"    → Prediction: {pred} ({'Fresh' if pred == 1 else 'Bad'}), Confidence: {prob:.2%}\n")

# Reset buffer for spoiled food test
buffer = SensorBuffer(buffer_size=SEQUENCE_LENGTH, scaler=scaler)

print("\nScenario: Spoiled food (high gas levels)")
for i in range(12):  # Add 12 readings
    # Simulate spoiled food readings with slight variation
    mq135 = np.random.randint(650, 700)
    mq3 = np.random.randint(720, 770)
    mics5524 = np.random.randint(680, 720)
    
    buffer.add_reading(mq135, mq3, mics5524)
    print(f"  Reading {i+1}: MQ135={mq135}, MQ3={mq3}, MiCS5524={mics5524}")
    
    # Try to predict once buffer is ready
    if buffer.is_ready():
        sequence = buffer.get_sequence()
        prob, pred = predict(sequence)
        print(f"    → Prediction: {pred} ({'Fresh' if pred == 1 else 'Bad'}), Confidence: {prob:.2%}\n")

## Summary

The LSTM model has been successfully loaded and tested with:
1. ✅ Random samples from the dataset
2. ✅ Custom fresh food sensor values
3. ✅ Custom spoiled food sensor values
4. ✅ Batch predictions with visualization
5. ✅ Real-time sensor buffer simulation (Arduino-like)

The model is ready for deployment!